* fineweb-edu
* mfu calc
* wandb logging

In [1]:
# !pip install tiktoken tqdm datasets wandb

In [2]:
import torch, random, numpy as np

vocab_size = 50_272
n_embd = 384
n_head = 6
n_layer = 6
dropout = 0.2
batch_size = 64
block_size = 128 
learning_rate = 3e-4

# seed
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# GPUs
gpus = {
    "M3": [4.1 * 10**12, "mps"],
    "4090": [82 * 10**12, "cuda"],
}
gpu = "M3"
flops_promised, device = gpus[gpu]

# device = "mps"
# flops_promised = 4.1 * 10**12 if device == "mps" else 82 * 10**12

In [3]:
from tqdm import tqdm
import tiktoken
import torch 
import datasets
import random

dataset = datasets.load_dataset("HuggingFaceFW/fineweb-edu", data_files=["sample/10BT/000_00000.parquet"], split="train")
dataset = dataset.train_test_split()

enc = tiktoken.get_encoding("gpt2")
assert enc.decode(enc.encode("hello world")) == "hello world"

def encode(string):
    return torch.tensor(enc.encode(string), dtype=torch.long)

def decode(tensor):
    return enc.decode(tensor.cpu().squeeze().numpy())

num_samples = 1_000
dataset_tok_train = torch.cat([encode(dataset["train"][i]["text"]) for i in tqdm(range(num_samples))])
dataset_tok_test = torch.cat([encode(dataset["test"][i]["text"]) for i in tqdm(range(num_samples))])

def get_sample(split, sample_length, batch_size):
    tokens = dataset_tok_train if split == "train" else dataset_tok_test
    idcs = torch.randint(len(tokens)-sample_length, (batch_size,))
    x = torch.stack([torch.tensor(tokens[x:x+sample_length]) for x in idcs])
    y = torch.stack([torch.tensor(tokens[x+1:x+sample_length+1]) for x in idcs])
    return x, y

100%|███████████████████████████████████████████████| 1000/1000 [00:00<00:00, 1466.33it/s]


In [4]:
print(f"Train data: {len(dataset_tok_train):,} tokens")
print(f"Test data: {len(dataset_tok_test):,} tokens")

Train data: 1,079,629 tokens
Test data: 964,951 tokens


In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F

class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))

        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        # input of size (batch, time-step, channels)
        # output of size (batch, time-step, head size)
        B,T,C = x.shape
        k = self.key(x)   # (B,T,hs)
        q = self.query(x) # (B,T,hs)
        # compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B, T, hs) @ (B, hs, T) -> (B, T, T)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf')) # (B, T, T)
        wei = F.softmax(wei, dim=-1) # (B, T, T)
        wei = self.dropout(wei)
        # perform the weighted aggregation of the values
        v = self.value(x) # (B,T,hs)
        # out = F.scaled_dot_product_attention(q, k, v)
        out = wei @ v # (B, T, T) @ (B, T, hs) -> (B, T, hs)
        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        out = torch.cat([h(x) for h in self.heads], dim=-1)
        out = self.dropout(self.proj(out))
        return out

class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

class GPTLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

        # better init, not covered in the original GPT video, but important, will cover in followup video
        self.apply(self._init_weights)

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

    def calculate_mfu(self, dt):
        flops_achieved = flops_per_fwdbwd / dt
        mfu = flops_achieved / flops_promised
        # print(f"{flops_achieved/10**12} TFLOPS achieved, {mfu * 100:.2f}% MFU")
        return mfu      

In [6]:
model = GPTLanguageModel()
m = model.to(device)
# print the number of parameters in the model
num_params = sum(p.numel() for p in m.parameters())/1e6
print(f"{num_params:.2f}M parameter model on device{device} with {flops_promised / 10**12:,} TFLOPS promised " )

import wandb 
wandb.init(
    project="minimal-llm",
    name=f"GPT2-{num_params:.1f}M-{len(dataset_tok_train)/10**6:,.1f}MT-{gpu}"
).log_code(".")

49.35M parameter model on devicemps with 4.1 TFLOPS promised 


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: gerald-stampfel. Use `wandb login --relogin` to force relogin


wandb: WARNING No relevant files were detected in the specified directory. No code will be logged to your run.


In [7]:
from torch.utils.flop_counter import FlopCounterMode

def get_flops(f):
    flop_counter = FlopCounterMode(display=False)
    with flop_counter:
        f()
    return flop_counter.get_total_flops() 

def train_one_sample():
    xb, yb = get_sample('train', block_size, batch_size)
    logits, loss = model(xb.to(device), yb.to(device))
    loss.backward()

flops_per_fwdbwd = get_flops(train_one_sample)
print(f"{flops_per_fwdbwd:,} flops per fwd+bwd pass")

/var/folders/fd/9xbk36z13vx8c3h6tbf6qchm0000gn/T/ipykernel_85162/1395468099.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.stack([torch.tensor(tokens[x:x+sample_length]) for x in idcs])
/var/folders/fd/9xbk36z13vx8c3h6tbf6qchm0000gn/T/ipykernel_85162/1395468099.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.stack([torch.tensor(tokens[x+1:x+sample_length+1]) for x in idcs])


28,991,029,248 flops per fwd+bwd pass


In [8]:
import time

log_interval = 10
eval_interval = 50
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

curr_time = time.time()
tok_total = 0 

for curr_step in range(1, 1000):
    # sample a batch of data
    xb, yb = get_sample('train', block_size, batch_size)
    tok_step = xb.view(-1).size(0)
    tok_total += tok_step

    # evaluate the loss
    step_start = time.time()
    logits, loss = model(xb.to(device), yb.to(device))
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

    step_time = time.time() - step_start
    
    if curr_step % log_interval == 0:
        mfu = model.calculate_mfu(step_time)
        print(f"step {curr_step}: loss {loss.detach().item():.2f} {step_time*1000:.2f}ms/step {(tok_step/step_time):,.0f}tok/s (total {tok_total:,} tok), {mfu * 100:.2f}% MFU")
        wandb.log({
            "step": curr_step,
            "tokens": tok_total,
            "loss_train": loss.detach().item(), 
            "mfu": mfu * 100, 
            "tokens/s": tok_step/step_time
        })

        
    if curr_step % eval_interval == 0:
        model.eval()
        xb, yb = get_sample('validation', block_size, batch_size)
        logits, loss = model(xb.to(device), yb.to(device))
        print(f"step {curr_step} eval: val_loss {loss.detach().item():.2f}")
        wandb.log({
            "step": curr_step,
            "tokens": tok_total,
            "loss_val": loss.detach().item()
        })
        # with torch.no_grad():
        #     output = model.generate(encode("hello my name is").to(device).unsqueeze(0), 10)
        #     print(decode(output))
        model.train()
        


/var/folders/fd/9xbk36z13vx8c3h6tbf6qchm0000gn/T/ipykernel_85162/1395468099.py:26: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = torch.stack([torch.tensor(tokens[x:x+sample_length]) for x in idcs])
/var/folders/fd/9xbk36z13vx8c3h6tbf6qchm0000gn/T/ipykernel_85162/1395468099.py:27: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.stack([torch.tensor(tokens[x+1:x+sample_length+1]) for x in idcs])


step 10: loss 9.43 1223.43ms/step 6,696tok/s (total 81,920 tok), 0.58% MFU
step 20: loss 8.50 1211.50ms/step 6,762tok/s (total 163,840 tok), 0.58% MFU
step 30: loss 7.94 1237.57ms/step 6,619tok/s (total 245,760 tok), 0.57% MFU
step 40: loss 7.67 1268.14ms/step 6,460tok/s (total 327,680 tok), 0.56% MFU
step 50: loss 7.59 1258.01ms/step 6,512tok/s (total 409,600 tok), 0.56% MFU
step 50 eval: val_loss 7.91
step 60: loss 7.61 1519.46ms/step 5,391tok/s (total 491,520 tok), 0.47% MFU
step 70: loss 7.54 3630.34ms/step 2,257tok/s (total 573,440 tok), 0.19% MFU
step 80: loss 7.38 2433.61ms/step 3,366tok/s (total 655,360 tok), 0.29% MFU
step 90: loss 7.28 2280.56ms/step 3,592tok/s (total 737,280 tok), 0.31% MFU
step 100: loss 7.24 1998.55ms/step 4,099tok/s (total 819,200 tok), 0.35% MFU
step 100 eval: val_loss 7.31
step 110: loss 7.21 2442.88ms/step 3,353tok/s (total 901,120 tok), 0.29% MFU
step 120: loss 6.99 1975.18ms/step 4,147tok/s (total 983,040 tok), 0.36% MFU
step 130: loss 7.00 2011.70ms

KeyboardInterrupt: 

In [ ]:
with torch.no_grad():
    output = model.generate(encode("This").to(device).unsqueeze(0), 50)
    print(decode(output))
